In [1]:
import pandas as pd 
from optbinning import OptimalBinning
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('gen_data.csv')

### Tính đầy đủ 

In [3]:
pd.set_option('display.max_rows', None)

Tỷ lệ missing theo cột

In [4]:
missing_col = pd.DataFrame({
    'missing_count': df.isna().sum(),
    'missing_pct': df.isna().mean() * 100
}).sort_values('missing_pct', ascending=False)

missing_col

,missing_count,missing_pct
SOCIF,0,0.0
PURCOD_MIN,0,0.0
FLAG_DEPOSIT,0,0.0
CBAL_SHORTTERM_LOAN,0,0.0
CBAL_LONGTERM_LOAN,0,0.0
HAS_SHORTTERM_LOAN,0,0.0
HAS_LONGTERM_LOAN,0,0.0
DURATION_MAX,0,0.0
REMAINING_DURATION_MAX,0,0.0
TIME_TO_OP_MAX,0,0.0


In [5]:
pd.set_option('display.max_rows', 20)

Tỷ lệ missing theo row

In [6]:
df['ROW_MISSING_PCT'] = df.isna().mean(axis=1) * 100

df['ROW_MISSING_PCT'].describe()

count    1138163.0
mean           0.0
std            0.0
min            0.0
25%            0.0
50%            0.0
75%            0.0
max            0.0
Name: ROW_MISSING_PCT, dtype: float64

In [7]:
df['FLAG_HIGH_MISSING'] = (df['ROW_MISSING_PCT'] > 40).astype(int)

### Tính duy nhất 

Trùng khóa logic (SOCIF – year)

In [8]:
dup_key = df.duplicated(subset=['SOCIF', 'year'], keep=False)

df.loc[dup_key].shape

(0, 53)

Trùng toàn bộ bản ghi (record duplicate)

In [9]:
dup_full = df.duplicated(keep=False)

dup_full.sum()


np.int64(0)

In [10]:
dup_rate = dup_full.mean() * 100
print(f"Tỷ lệ bản ghi trùng lặp: {dup_rate:.2f}%")


Tỷ lệ bản ghi trùng lặp: 0.00%


### Tính kịp thời 

Check dùng future information (leakage)

In [11]:
# BAD_NEXT_12M chỉ được missing ở năm cuối
leak_check = df[
    (df['year'] < df['year'].max()) &
    (df['BAD_NEXT_12M'].isna())
]

leak_check.shape


(0, 53)

Check continuity theo năm

In [12]:
year_count = df.groupby('year')['SOCIF'].nunique()
year_count


year
2018    199924
2019    204308
2020    155886
2021    160132
2022    206744
2023    211169
Name: SOCIF, dtype: int64

### Tính phù hợp 

In [13]:
rule_valid_age = (
    df['TUOI'].between(0, 120)
)

df.loc[~rule_valid_age].shape


(0, 53)

In [14]:
rule_valid_ltv = df['LTV'].between(0, 400)

df.loc[~rule_valid_ltv].shape


(0, 53)

In [15]:
rule_valid_duration = (
    (df['DURATION_MAX'] >= 0) &
    (df['REMAINING_DURATION_MAX'] >= 0)
)

df.loc[~rule_valid_duration].shape


(0, 53)

In [ ]:
valid_gender = df['C_GIOITINH'].isin(['M', 'F', 'O'])
df.loc[~valid_gender].shape

(0, 53)

In [ ]:
df['FLAG_INVALID'] = ~(
    rule_valid_age &
    rule_valid_ltv &
    rule_valid_duration &
    valid_gender
)

In [18]:
pd.set_option('display.max_columns', None)

In [19]:
df.describe()

,SOCIF,TRINHDO,SOHUUNHA,NHANVIENBIDV,INHERENT_RISK,year,BASE_AUM,CURRENT_RISK,TUOI,INCOME,CBAL,CBALORG,AFLIMT_MAX,AFLIMT_MIN,AFLIMT_AVG,CBAL_AVG,CBAL_MAX,CBAL_MIN,COLLATERAL_VALUE,LTV,N_AVG_DEPOSIT_12M,N_AVG_DEPOSIT_6M,N_AVG_DD_12M,N_AVG_CD_12M,FLAG_SALARY_ACC,FLAG_DEPOSIT,CBAL_SHORTTERM_LOAN,CBAL_LONGTERM_LOAN,HAS_SHORTTERM_LOAN,HAS_LONGTERM_LOAN,DURATION_MAX,REMAINING_DURATION_MAX,TIME_TO_OP_MAX,RATE_AVG,PURCOD_MAX,PURCOD_MIN,MAX_DPD_12M,MAX_DPD_12M_OBS,AVG_OD_DPD_12M,SUM_ALL_OD_12M,BAD_CURRENT,XULYNO,MAX_NHOMNOCIC,N_AVG_OVERDUE_CBAL_12M,CBAL_TO_INC_12MON,REAL_GDP_GROWTH_12M,BAD_NEXT_12M,ROW_MISSING_PCT,FLAG_HIGH_MISSING
count,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1138163.0,1138163.0,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1138163.0,1138163.0,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1.138163e+06,1138163.0,1138163.0
mean,1.200013e+06,1.963288e+00,3.700964e-01,9.951123e-03,-4.511968e-01,2.020530e+03,1.556609e+07,-4.227742e-01,3.873186e+01,2.464436e+07,3.569170e+08,3.926050e+08,7.711901e+08,6.940711e+08,7.326306e+08,3.568099e+08,4.287724e+08,2.854480e+08,7.526456e+08,1.326585e+02,5.600610e+06,5.600610e+06,1.120122e+06,4.480488e+06,9.126979e-01,8.279271e-01,1.784538e+08,1.784632e+08,1.0,1.0,9.138879e+01,4.748012e+01,4.390867e+01,8.211669e+00,4.599542e+00,4.599542e+00,2.444966e+01,2.444966e+01,6.870292e+00,4.840832e+01,0.0,0.0,1.618325e+00,1.666181e+08,1.442905e+01,5.659087e+00,1.209642e-01,0.0,0.0
std,1.154080e+05,8.242566e-01,4.828305e-01,9.925778e-02,6.118998e-01,1.771599e+00,2.277483e+07,7.558544e-01,9.690110e+00,1.034253e+07,2.544244e+08,2.809715e+08,3.581232e+08,3.223108e+08,3.402170e+08,2.593033e+08,3.113689e+08,2.074426e+08,9.676977e+08,8.801497e+01,9.881189e+06,9.881189e+06,1.976238e+06,7.904951e+06,2.822773e-01,3.774440e-01,1.795426e+08,1.792469e+08,0.0,0.0,6.661096e+01,4.462007e+01,4.207698e+01,2.021614e+00,2.537987e+00,2.537987e+00,2.407122e+01,2.407122e+01,8.210826e+00,5.200160e+01,0.0,0.0,4.857976e-01,1.309829e+08,7.527281e+00,2.034061e+00,3.260858e-01,0.0,0.0
min,1.000000e+06,1.000000e+00,0.000000e+00,0.000000e+00,-1.414093e+00,2.018000e+03,6.741400e+04,-3.449609e+00,1.800000e+01,9.172115e+06,2.040050e+05,2.347340e+05,2.303135e+08,2.072821e+08,2.187978e+08,1.735020e+05,2.082020e+05,1.388010e+05,8.009100e+04,5.756351e-02,1.065600e+04,1.065600e+04,2.131000e+03,8.525000e+03,0.000000e+00,0.000000e+00,2.030000e+02,2.290000e+02,1.0,1.0,6.000000e+00,0.000000e+00,1.000000e+00,5.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,1.000000e+00,0.000000e+00,6.722137e-03,2.580000e+00,0.000000e+00,0.0,0.0
25%,1.100321e+06,1.000000e+00,0.000000e+00,0.000000e+00,-9.132219e-01,2.019000e+03,4.346756e+06,-9.470041e-01,3.200000e+01,1.815152e+07,1.874383e+08,2.056924e+08,5.466358e+08,4.919723e+08,5.193041e+08,1.848549e+08,2.222427e+08,1.478840e+08,4.777883e+06,2.019611e+01,1.350348e+06,1.350348e+06,2.700690e+05,1.080279e+06,1.000000e+00,1.000000e+00,5.897268e+07,5.870722e+07,1.0,1.0,3.600000e+01,1.700000e+01,1.500000e+01,6.703383e+00,2.000000e+00,2.000000e+00,5.000000e+00,5.000000e+00,1.000000e+00,8.000000e+00,0.0,0.0,1.000000e+00,8.307394e+07,8.751274e+00,2.910000e+00,0.000000e+00,0.0,0.0
50%,1.199950e+06,2.000000e+00,0.000000e+00,0.000000e+00,-5.664156e-01,2.021000e+03,8.792217e+06,-4.678683e-01,3.900000e+01,2.216280e+07,2.935441e+08,3.224625e+08,6.826361e+08,6.143725e+08,6.485043e+08,2.913740e+08,3.501939e+08,2.330992e+08,1.480973e+07,2.000000e+02,2.837548e+06,2.837548e+06,5.675090e+05,2.270039e+06,1.000000e+00,1.000000e+00,1.278065e+08,1.278327e+08,1.0,1.0,6.000000e+0

In [20]:
pd.set_option('display.max_columns', 20)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138163 entries, 0 to 1138162
Data columns (total 54 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   SOCIF                   1138163 non-null  int64  
 1   C_GIOITINH              1138163 non-null  object 
 2   TRINHDO                 1138163 non-null  int64  
 3   TTHONNHAN               1138163 non-null  object 
 4   SOHUUNHA                1138163 non-null  int64  
 5   NHANVIENBIDV            1138163 non-null  int64  
 6   INHERENT_RISK           1138163 non-null  float64
 7   year                    1138163 non-null  int64  
 8   BASE_AUM                1138163 non-null  int64  
 9   CURRENT_RISK            1138163 non-null  float64
 10  TUOI                    1138163 non-null  int64  
 11  SNAPSHOT_DATE           1138163 non-null  object 
 12  INCOME                  1138163 non-null  int64  
 13  CBAL                    1138163 non-null  int64  
 14  CB

In [22]:
cat_cols = df.select_dtypes(include=['object', 'category']).columns

for col in cat_cols:
    print(f"\n{'='*20} {col} {'='*20}")
    vc = df[col].value_counts(dropna=False)
    pct = df[col].value_counts(normalize=True, dropna=False) * 100
    print(pd.concat([vc, pct.rename('pct_%')], axis=1))


==================== C_GIOITINH ====================
             count      pct_%
C_GIOITINH                   
F           560176  49.217555
M           555388  48.796877
O            22599   1.985568

==================== TTHONNHAN ====================
            count      pct_%
TTHONNHAN                   
Married    596074  52.371585
Single     542089  47.628415

==================== SNAPSHOT_DATE ====================
               count     pct_%
SNAPSHOT_DATE                 
2023-06-28      1258  0.110529
2022-06-26      1237  0.108684
2019-03-15      1236  0.108596
2019-02-10      1216  0.106839
2019-02-05      1213  0.106575
...              ...       ...
2021-01-03        26  0.002284
2021-10-06        25  0.002197
2021-10-08        23  0.002021
2020-01-03        23  0.002021
2020-01-04        20  0.001757

[1575 rows x 2 columns]

==================== SAMPLE_TYPE ====================
              count      pct_%
SAMPLE_TYPE                   
TRAIN        720250  63.2

### Tính chính xác

Tuổi khách hàng

In [23]:
rule_acc_age = df['TUOI'] >= 15

LTV thực tế

In [24]:
rule_acc_ltv = df['LTV'] < 400

Hạn mức – dư nợ

In [25]:
rule_acc_limit = df['AFLIMT_MAX'] >= df['CBAL']

Thu nhập – dư nợ

In [26]:
rule_acc_dti = df['CBAL'] <= df['INCOME'] * 20

CIC – DPD

In [27]:
rule_acc_cic = ~(
    (df['MAX_DPD_12M_OBS'] < 30) & (df['MAX_NHOMNOCIC'] > 1)
) & ~(
    (df['MAX_DPD_12M_OBS'] >= 90) & (df['MAX_NHOMNOCIC'] < 3)
)

Flag & tỷ lệ chính xác

In [28]:
df['FLAG_INACCURATE'] = ~(
    rule_acc_age &
    rule_acc_ltv &
    rule_acc_limit &
    rule_acc_dti &
    rule_acc_cic
)

accuracy_rate = 1 - df['FLAG_INACCURATE'].mean()
accuracy_rate

np.float64(0.565151915850366)

In [29]:
rule_acc_age.sum()

np.int64(1138163)

In [30]:
rule_acc_ltv.sum()

np.int64(1138163)

In [31]:
rule_acc_dti.sum()

np.int64(902421)

In [32]:
rule_acc_cic.sum()

np.int64(810422)

### Tính đồng nhất

CBAL vs cấu trúc kỳ hạn

In [33]:
# rule_cons_balance = (
#     df['CBAL'] ==
#     df['CBAL_SHORTTERM_LOAN']
#     + df['CBAL_MIDTERM_LOAN']
#     + df['CBAL_LONGTERM_LOAN']
# )

# df.loc[~rule_cons_balance].shape

Deposit breakdown

In [34]:
rule_cons_deposit = (
    abs(
        df['N_AVG_DEPOSIT_12M'] -
        (df['N_AVG_DD_12M'] + df['N_AVG_CD_12M'])
    ) <= 1
)
df.loc[~rule_cons_deposit].shape

(0, 55)

DPD logic

In [35]:
rule_cons_dpd = df['MAX_DPD_12M_OBS'] <= df['MAX_DPD_12M']

df.loc[~rule_cons_dpd].shape


(0, 55)

Flag tổng hợp consistency

In [36]:
df['FLAG_INCONSISTENT'] = ~(
    # rule_cons_balance &
    rule_cons_deposit &
    rule_cons_dpd
)

BẢNG TỔNG HỢP (RẤT NÊN CÓ)

In [37]:
quality_summary = pd.DataFrame({
    'Metric': [
        'Completeness',
        'Uniqueness',
        'Timeliness',
        'Validity',
        'Accuracy',
        'Consistency'
    ],
    'Issue_Rate_%': [
        df['FLAG_HIGH_MISSING'].mean() * 100,
        dup_full.mean() * 100,
        leak_check.shape[0] / len(df) * 100,
        df['FLAG_INVALID'].mean() * 100,
        df['FLAG_INACCURATE'].mean() * 100,
        df['FLAG_INCONSISTENT'].mean() * 100
    ]
})

quality_summary

,Metric,Issue_Rate_%
0,Completeness,0.000000
1,Uniqueness,0.000000
2,Timeliness,0.000000
3,Validity,0.000000
4,Accuracy,43.484808
5,Consistency,0.000000


In [38]:
df['FLAG_AGE_INVALID'] = (~rule_acc_age).astype(int)
df['FLAG_LTV_HIGH']    = (~rule_acc_ltv).astype(int)
df['FLAG_DTI_HIGH']    = (~rule_acc_dti).astype(int)
df['FLAG_CIC_LOW']     = (~rule_acc_cic).astype(int)
# df['FLAG_INCONSISTENT_DEPOSIT'] = (~rule_cons_deposit).astype(int)

In [39]:
import numpy as np

In [40]:
flag_cols = [c for c in df.columns if c.startswith('FLAG_')]

dq_flag_impact = []

for c in flag_cols:
    tmp = df.groupby(c)['BAD_NEXT_12M'].mean()
    dq_flag_impact.append({
        'FLAG': c,
        'BAD_RATE_0': tmp.get(0, np.nan),
        'BAD_RATE_1': tmp.get(1, np.nan),
        'LIFT': tmp.get(1, np.nan) / tmp.get(0, np.nan)
    })

dq_flag_impact = pd.DataFrame(dq_flag_impact).sort_values('LIFT', ascending=False)
dq_flag_impact

,FLAG,BAD_RATE_0,BAD_RATE_1,LIFT
8,FLAG_DTI_HIGH,0.113921,0.147924,1.298479
4,FLAG_INACCURATE,0.138997,0.097527,0.701648
0,FLAG_SALARY_ACC,0.174248,0.115867,0.664956
9,FLAG_CIC_LOW,0.148419,0.053075,0.357606
1,FLAG_DEPOSIT,0.282149,0.087464,0.309993
2,FLAG_HIGH_MISSING,0.120964,NaN,NaN
3,FLAG_INVALID,0.120964,NaN,NaN
5,FLAG_INCONSISTENT,0.120964,NaN,NaN
6,FLAG_AGE_INVALID,0.120964,NaN,NaN
7,FLAG_LTV_HIGH,0.120964,NaN,NaN


In [41]:
for f in ['FLAG_HIGH_MISSING', 'FLAG_INCONSISTENT']:
    print(f, df[f].value_counts(dropna=False))


FLAG_HIGH_MISSING FLAG_HIGH_MISSING
0    1138163
Name: count, dtype: int64
FLAG_INCONSISTENT FLAG_INCONSISTENT
False    1138163
Name: count, dtype: int64


In [42]:
df.shape

(1138163, 60)

In [43]:
df.drop(['FLAG_DTI_HIGH', 'FLAG_INACCURATE', 'FLAG_CIC_LOW', 'FLAG_HIGH_MISSING', 'FLAG_INCONSISTENT', 'ROW_MISSING_PCT'], axis=1, inplace=True)

In [ ]:
df.columns

Index(['SOCIF', 'C_GIOITINH', 'TRINHDO', 'TTHONNHAN', 'SOHUUNHA',
       'NHANVIENBIDV', 'INHERENT_RISK', 'year', 'BASE_AUM', 'CURRENT_RISK',
       'TUOI', 'SNAPSHOT_DATE', 'INCOME', 'CBAL', 'CBALORG', 'AFLIMT_MAX',
       'AFLIMT_MIN', 'AFLIMT_AVG', 'CBAL_AVG', 'CBAL_MAX', 'CBAL_MIN',
       'COLLATERAL_VALUE', 'LTV', 'N_AVG_DEPOSIT_12M', 'N_AVG_DEPOSIT_6M',
       'N_AVG_DD_12M', 'N_AVG_CD_12M', 'FLAG_SALARY_ACC', 'FLAG_DEPOSIT',
       'CBAL_SHORTTERM_LOAN', 'CBAL_LONGTERM_LOAN', 'HAS_SHORTTERM_LOAN',
       'HAS_LONGTERM_LOAN', 'DURATION_MAX', 'REMAINING_DURATION_MAX',
       'TIME_TO_OP_MAX', 'RATE_AVG', 'PURCOD_MAX', 'PURCOD_MIN', 'MAX_DPD_12M',
       'MAX_DPD_12M_OBS', 'AVG_OD_DPD_12M', 'SUM_ALL_OD_12M', 'BAD_CURRENT',
       'XULYNO', 'MAX_NHOMNOCIC', 'N_AVG_OVERDUE_CBAL_12M',
       'CBAL_TO_INC_12MON', 'REAL_GDP_GROWTH_12M', 'BAD_NEXT_12M',
       'SAMPLE_TYPE', 'FLAG_INVALID', 'FLAG_AGE_INVALID', 'FLAG_LTV_HIGH'],
      dtype='object')

In [45]:
df_train = df[df['SAMPLE_TYPE'] == 'TRAIN']
df_train_y = df_train['BAD_NEXT_12M']
df_OOS = df[df['SAMPLE_TYPE'] == 'OOS']
df_OOS_y = df_OOS['BAD_NEXT_12M']
df_OOT = df[df['SAMPLE_TYPE'] == 'OOT']
df_OOT_y = df_OOT['BAD_NEXT_12M']

In [46]:
df_train.drop(['SAMPLE_TYPE','BAD_NEXT_12M'], axis=1, inplace=True)
df_OOS.drop(['SAMPLE_TYPE', 'BAD_NEXT_12M'], axis=1, inplace=True)
df_OOT.drop(['SAMPLE_TYPE', 'BAD_NEXT_12M'], axis=1, inplace=True)

In [47]:
df_train.columns

Index(['SOCIF', 'C_GIOITINH', 'TRINHDO', 'TTHONNHAN', 'SOHUUNHA',
       'NHANVIENBIDV', 'INHERENT_RISK', 'year', 'BASE_AUM', 'CURRENT_RISK',
       'TUOI', 'SNAPSHOT_DATE', 'INCOME', 'CBAL', 'CBALORG', 'AFLIMT_MAX',
       'AFLIMT_MIN', 'AFLIMT_AVG', 'CBAL_AVG', 'CBAL_MAX', 'CBAL_MIN',
       'COLLATERAL_VALUE', 'LTV', 'N_AVG_DEPOSIT_12M', 'N_AVG_DEPOSIT_6M',
       'N_AVG_DD_12M', 'N_AVG_CD_12M', 'FLAG_SALARY_ACC', 'FLAG_DEPOSIT',
       'CBAL_SHORTTERM_LOAN', 'CBAL_LONGTERM_LOAN', 'HAS_SHORTTERM_LOAN',
       'HAS_LONGTERM_LOAN', 'DURATION_MAX', 'REMAINING_DURATION_MAX',
       'TIME_TO_OP_MAX', 'RATE_AVG', 'PURCOD_MAX', 'PURCOD_MIN', 'MAX_DPD_12M',
       'MAX_DPD_12M_OBS', 'AVG_OD_DPD_12M', 'SUM_ALL_OD_12M', 'BAD_CURRENT',
       'XULYNO', 'MAX_NHOMNOCIC', 'N_AVG_OVERDUE_CBAL_12M',
       'CBAL_TO_INC_12MON', 'REAL_GDP_GROWTH_12M', 'FLAG_INVALID',
       'FLAG_AGE_INVALID', 'FLAG_LTV_HIGH'],
      dtype='object')